# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# # checking your current working directory
# print(os.getcwd())

# # Get your current folder and subfolder event data
# filepath = os.getcwd() + '/event_data'

# # Create a for loop to create a list of files and collect each filepath
# for root, dirs, files in os.walk(filepath):
    
# # join the file path and roots with the subdirectories using glob
#     file_path_list = glob.glob(os.path.join(root,'*'))
#     #print(file_path_list)
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root, '*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
# full_data_rows_list = [] 
    
# for every filepath in the file path list 
# for f in file_path_list:

# # reading csv file 
#     with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
#         # creating a csv reader object 
#         csvreader = csv.reader(csvfile) 
#         next(csvreader)
        
#  # extracting each data row one by one and append it        
#         for line in csvreader:
#             #print(line)
#             full_data_rows_list.append(line) 
            
# # uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# # uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# # Apache Cassandra tables
# csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
#     writer = csv.writer(f, dialect='myDialect')
#     writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
#                 'level','location','sessionId','song','userId'])
#     for row in full_data_rows_list:
#         if (row[0] == ''):
#             continue
#         writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

full_data_rows_list = []
for f in file_path_list:
    #reading csv files
    with open(f, 'r', encoding = 'utf8', newline = '') as csvfile:
        #create a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line)

print(len(full_data_rows_list))
#print(full_data_rows_list)
#Create a smaller csv file to insert data into the Apache Cassandra tables

csv.register_dialect('OLIDialect', quoting = csv.QUOTE_ALL, skipinitialspace = True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline = '') as f:
    writer = csv.writer(f, dialect = 'OLIDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                  'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

        


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
query = """
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1 - Create Table

In [12]:
query = """
CREATE TABLE IF NOT EXISTS song_info_session_item
(session_id int,
item_in_session int,
artist text,
song_title text,
song_length float,
PRIMARY KEY (session_id, item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

#### Query 1 - Insert Data

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO song_info_session_item (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query 1 - Do a SELECT to verify that the data have been inserted into each table

In [16]:
query = """
SELECT artist, song_title, song_length
FROM song_info_session_item
WHERE session_id = 338
AND item_in_session = 4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2 - Create table

In [18]:
query = """
CREATE TABLE IF NOT EXISTS song_info_by_user
(user_id int,
session_id int,
item_in_session int,
artist text,
song_title text,
user_full_name text,
PRIMARY KEY ((user_id, session_id), item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

#### Query 2 - Insert Data

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_user (user_id, session_id, item_in_session, artist, song_title, user_full_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1] + ' ' + line[4])))

#### Query 2 - Do a SELECT to verify that the data have been inserted into each table

In [20]:
query = """
SELECT artist, item_in_session, song_title, user_full_name
FROM song_info_by_user
WHERE user_id = 10
AND session_id = 182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    #I am adding the item_in_session to verify if the sorting with item_in_session is correct
    print(row.artist, row.item_in_session, row.song_title, row.user_full_name)


Down To The Bone 0 Keep On Keepin' On Sylvie Cruz
Three Drives 1 Greece 2000 Sylvie Cruz
Sebastien Tellier 2 Kilometer Sylvie Cruz
Lonnie Gordon 3 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3 - Create Table

In [21]:
query = """
CREATE TABLE IF NOT EXISTS song_name_by_user
(song_title text,
user_id int,
user_full_name text,
PRIMARY KEY (song_title, user_id))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

#### Query 3 - Insert Data

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_name_by_user (song_title, user_id, user_full_name)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), (line[1] + ' ' + line[4])))

#### Query 3 - Do a SELECT to verify that the data have been inserted into each table

In [25]:
query = """
SELECT user_full_name
FROM song_name_by_user
WHERE song_title = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.user_full_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Load the cleaned csv file into DataFrame to validate results

In [50]:
#Using below code to validate the results
df = pd.read_csv(file)
df['full_name'] = df.apply(lambda row: row.firstName + " " + row.lastName, axis = 1)
print("Total rows of the cleaned csv file is", df.shape[0])

#Validate Query 1

df1 = df[(df['sessionId'] == 338) & (df['itemInSession'] == 4)]
print("\nResults for query 1:")
print(df1[['artist', 'song', 'length']])

#Validate Query 2
df2 = df[(df['sessionId'] == 182) & (df['userId'] == 10)]
print("\nResults for query 2:")
print(df2[['artist', 'song', 'full_name']])

df3 = df.loc[df['song'] == 'All Hands Against His Own']
print("\nResults for query 3:")
print(df3['full_name'])


Total rows of the cleaned csv file is 6820

Results for query 1:
        artist                             song    length
444  Faithless  Music Matters (Mark Knight Dub)  495.3073

Results for query 2:
                 artist                                               song  \
4704   Down To The Bone                                 Keep On Keepin' On   
4705       Three Drives                                        Greece 2000   
4706  Sebastien Tellier                                          Kilometer   
4707      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

        full_name  
4704  Sylvie Cruz  
4705  Sylvie Cruz  
4706  Sylvie Cruz  
4707  Sylvie Cruz  

Results for query 3:
2792        Tegan Levine
5135        Sara Johnson
6298    Jacqueline Lynch
Name: full_name, dtype: object


### Checking the combining process of raw data files using pandas

In [51]:
#also doing this makes me realize the small csv file created at the beginning is filtering out the rows without the artist
df_list = (pd.read_csv(file) for file in file_path_list)
df = pd.concat(df_list, ignore_index=True)
df = df[df['artist'].notnull()]
df.shape


(6820, 17)

### Drop the tables before closing out the sessions

In [52]:
query = "DROP TABLE IF EXISTS event_table"
query1 = query + "1"
query2 = query + "2"
query3 = query + "3"
try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)



### Close the session and cluster connection¶

In [53]:
session.shutdown()
cluster.shutdown()